In [1]:
# Import all needed libraries
import pandas as pd
import numpy as np
import geopy
from geopy.distance import geodesic

In [19]:
# Read in the Fixed Radial csv file
nonstd = pd.read_csv('C:/Users/Olga/Desktop/GMU/DAEN690/LEGO/Non-Std-148FixedRadial.csv')
nonstd

,REMARKS,dateonly,timeonly,DATE,CALLSIGN,POD,PRIMARYCODE,SECONDARYCODES,REPORTINGFACILITY,MORID.CEDAR,...,ACALTITUDE.CEDAR,ACHEADING.CEDAR,RELATIVECLOCKPOSITION.CEDAR,UASLONG.CEDAR,UASLAT.CEDAR,SUMMARY.CEDAR,QAFINDINGS.CEDAR,RWYLOCATION,UASLOCATION,DATASET
0,"Small uas on the BWI185025, 7,500 feet, headin...",1/21/2018,0:39,2018-01-21T00:39Z,SWA3876,NCRCC,UAS,NaN,NCRCC,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,025 NM 185 of BWI,FIXED RADIAL
1,1756HELO departed CHS as VRTICL3. 1800HELOdepa...,2/15/2018,0:02,2018-02-15T00:02Z,N337H,DEN,UAS,MOR,CHS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,008 NM 130 of CHS,FIXED RADIAL
2,"uas: Pilot reported a red uas at altitude 5,50...",3/4/2018,20:46,2018-03-04T20:46Z,UAL1067,NCRCC,TOI-NCR SFRA,"PAGED, TOI, UAS",PCT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,028 NM 180 of DCA,FIXED RADIAL
3,DCA-1: White House at condition Yellow. Report...,3/31/2018,21:59,2018-03-31T21:59Z,NaN,NCRCC,TOI-PA-P-56A,"MOR, PAGED, TOI, TOI-PA, UAS",USSS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,586 NM 100 of CFR,FIXED RADIAL
4,"PIC reported a uas, black, 2-3 feet in diamete...",4/22/2018,1:48,2018-04-22T01:48Z,SWA4402,NCRCC,UAS,PAGED,PCT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,026 NM 267 of DCA,FIXED RADIAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,DCA-1: uas observed at the DCA283013 just insi...,7/7/2021,14:26,2021-07-07T14:26Z,NaN,NCRCC,UAS,NaN,NCRCC,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,013 NM 283 of DCA,FIXED RADIAL
144,"DCA-1: DCA355001, aircraft heading S Ebound re...",7/21/2021,3:44,2021-07-21T03:44Z,JIA5596,NCRCC,TOI-NCR FRZ,"MOR, UAS",DCA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,001 NM 355 of DCA,FIXED RADIAL
145,DCA-1: ADW Base Police observed a uas with red...,7/23/2021,6:45,2021-07-23T06:45Z,NaN,NCRCC,UAS,"MOR, SUSPICIOUS ACTIVITY, TOI-NCR FRZ",ADW,1.392354e+09,...,200.0,180.0,NaN,NaN,NaN,AT APPROXIMATELY 0637Z ADW'S UAS COUNTERMEASUR...,Facility summary reviewed. At approximately 06...,NaN,009 NM 121 of DCA,FIXED RADIAL
146,DCA-1: uas observed over Madison Building (DCA...,7/30/2021,23:50,2021-07-30T23:50Z,NaN,NCRCC,UAS,TOI-PA-P-56A,USCP,3.531522e+08,...,NaN,NaN,NaN,NaN,NaN,"US CAPITOL POLICE REPORTED UAS, DARK COLORED N...",MOR reviewed. DEN and law enforcement notified...,NaN,002 NM 052 of DCA,FIXED RADIAL


In [20]:
# Extract the IDENT from the UAS location column
uas_loc = nonstd['UASLOCATION'].tolist()
uas_airport = []

for i in range(len(uas_loc)):
    airport = uas_loc[i].split(' ')[-1]
    
    if len(airport) <= 4:
        uas_airport.append(airport)
    else:
        trim_air = airport[-3:]
        uas_airport.append(trim_air)
# Create a new dataframe that contains all IDENTs
uas_airport_df = pd.DataFrame()
uas_airport_df['IDENT'] = uas_airport
uas_airport_df

,IDENT
0,BWI
1,CHS
2,DCA
3,CFR
4,DCA
...,...
143,DCA
144,DCA
145,DCA
146,DCA


In [21]:
# Import the cleaned airports dataset
airports = pd.read_csv('C:/Users/Olga/Desktop/GMU/DAEN690/LEGO/airports_cleaned.csv')

In [22]:
# Merge and link the extracted IDENTs with the airports dataset to get airports' latitude and longitude
uas_air_loc= pd.merge(uas_airport_df, airports, on='IDENT', how='left')
uas_air_loc

,IDENT,NAME,LATITUDE,LONGITUDE,ICAO_ID
0,BWI,Baltimore/Washington Intl Thurgood Marshall,39.175728,-76.668991,KBWI
1,CHS,Charleston AFB/Intl,32.898639,-80.040528,KCHS
2,DCA,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA
3,CFR,NaN,NaN,NaN,NaN
4,DCA,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA
...,...,...,...,...,...
143,DCA,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA
144,DCA,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA
145,DCA,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA
146,DCA,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA


In [23]:
# Add the Remarks and UAS location columns
add_col = nonstd['REMARKS'].tolist()
uas_air_loc['REMARKS'] = add_col
add_col1 = nonstd['UASLOCATION'].tolist()
uas_air_loc['UASLOCATION'] = add_col1
uas_air_loc

,IDENT,NAME,LATITUDE,LONGITUDE,ICAO_ID,REMARKS,UASLOCATION
0,BWI,Baltimore/Washington Intl Thurgood Marshall,39.175728,-76.668991,KBWI,"Small uas on the BWI185025, 7,500 feet, headin...",025 NM 185 of BWI
1,CHS,Charleston AFB/Intl,32.898639,-80.040528,KCHS,1756HELO departed CHS as VRTICL3. 1800HELOdepa...,008 NM 130 of CHS
2,DCA,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA,"uas: Pilot reported a red uas at altitude 5,50...",028 NM 180 of DCA
3,CFR,NaN,NaN,NaN,NaN,DCA-1: White House at condition Yellow. Report...,586 NM 100 of CFR
4,DCA,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA,"PIC reported a uas, black, 2-3 feet in diamete...",026 NM 267 of DCA
...,...,...,...,...,...,...,...
143,DCA,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA,DCA-1: uas observed at the DCA283013 just insi...,013 NM 283 of DCA
144,DCA,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA,"DCA-1: DCA355001, aircraft heading S Ebound re...",001 NM 355 of DCA
145,DCA,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA,DCA-1: ADW Base Police observed a uas with red...,009 NM 121 of DCA
146,DCA,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA,DCA-1: uas observed over Madison Building (DCA...,002 NM 052 of DCA


In [24]:
# Find the records that did not match and save them in different csv file
uas_air_loc1 = uas_air_loc[uas_air_loc['NAME'].isnull()]
uas_air_loc1

,IDENT,NAME,LATITUDE,LONGITUDE,ICAO_ID,REMARKS,UASLOCATION
3,CFR,NaN,NaN,NaN,NaN,DCA-1: White House at condition Yellow. Report...,586 NM 100 of CFR
17,SIE,NaN,NaN,NaN,NaN,PIC reported uas off it's left wing at same al...,012 NM 330 of SIE
31,VAN,NaN,NaN,NaN,NaN,PIC reported uas off left side at FL250. No ev...,045 NM 091 of VAN
56,BRV,NaN,NaN,NaN,NaN,Notified that Stafford County Sheriff's Office...,007 NM 005 of BRV
89,LIN,NaN,NaN,NaN,NaN,"2056Z, FDC NOTAM 0/1240, The Quarter Fire TFR,...",033 NM 072 of LIN
138,XCH,NaN,NaN,NaN,NaN,"Navy Yard, DC reports a uas detection only ins...",030 NM 940 of XCH
139,XEB,NaN,NaN,NaN,NaN,Navy Yard reports detection systems at NSF Arl...,201 NM 100 of XEB


In [32]:
uas_air_loc1.to_csv('7FixedRadial.csv', index = False)

In [25]:
# Continue working with the matching records
uas_air_loc = uas_air_loc[uas_air_loc['NAME'].notnull()]
uas_air_loc

,IDENT,NAME,LATITUDE,LONGITUDE,ICAO_ID,REMARKS,UASLOCATION
0,BWI,Baltimore/Washington Intl Thurgood Marshall,39.175728,-76.668991,KBWI,"Small uas on the BWI185025, 7,500 feet, headin...",025 NM 185 of BWI
1,CHS,Charleston AFB/Intl,32.898639,-80.040528,KCHS,1756HELO departed CHS as VRTICL3. 1800HELOdepa...,008 NM 130 of CHS
2,DCA,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA,"uas: Pilot reported a red uas at altitude 5,50...",028 NM 180 of DCA
4,DCA,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA,"PIC reported a uas, black, 2-3 feet in diamete...",026 NM 267 of DCA
5,DCA,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA,uas operations DCA271012. Operations terminate...,012 NM 271 of DCA
...,...,...,...,...,...,...,...
143,DCA,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA,DCA-1: uas observed at the DCA283013 just insi...,013 NM 283 of DCA
144,DCA,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA,"DCA-1: DCA355001, aircraft heading S Ebound re...",001 NM 355 of DCA
145,DCA,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA,DCA-1: ADW Base Police observed a uas with red...,009 NM 121 of DCA
146,DCA,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA,DCA-1: uas observed over Madison Building (DCA...,002 NM 052 of DCA


In [26]:
# Save the records into separate csv file
uas_air_loc.to_csv('141FixedRadial.csv', index = False)
uas_air_loc = pd.read_csv('C:/Users/Olga/Desktop/GMU/DAEN690/LEGO/141FixedRadial.csv')
uas_air_loc

,IDENT,NAME,LATITUDE,LONGITUDE,ICAO_ID,REMARKS,UASLOCATION
0,BWI,Baltimore/Washington Intl Thurgood Marshall,39.175728,-76.668991,KBWI,"Small uas on the BWI185025, 7,500 feet, headin...",025 NM 185 of BWI
1,CHS,Charleston AFB/Intl,32.898639,-80.040528,KCHS,1756HELO departed CHS as VRTICL3. 1800HELOdepa...,008 NM 130 of CHS
2,DCA,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA,"uas: Pilot reported a red uas at altitude 5,50...",028 NM 180 of DCA
3,DCA,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA,"PIC reported a uas, black, 2-3 feet in diamete...",026 NM 267 of DCA
4,DCA,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA,uas operations DCA271012. Operations terminate...,012 NM 271 of DCA
...,...,...,...,...,...,...,...
136,DCA,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA,DCA-1: uas observed at the DCA283013 just insi...,013 NM 283 of DCA
137,DCA,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA,"DCA-1: DCA355001, aircraft heading S Ebound re...",001 NM 355 of DCA
138,DCA,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA,DCA-1: ADW Base Police observed a uas with red...,009 NM 121 of DCA
139,DCA,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA,DCA-1: uas observed over Madison Building (DCA...,002 NM 052 of DCA


In [27]:
# Separate the distance and bearing into two different columns
uas_loc = uas_air_loc['UASLOCATION']
distances = []
bearings = []

for i in range(len(uas_loc)):
    split = uas_loc[i].split(' ')[0:3]
    
    if 'NM' in split[0]: 
        distance_nm = split[0][0:len(split[0]) - 2]
        bearing_abrev = split[1].replace('of', '')
        
    else: 
        distance_nm = split[0]
        bearing_abrev = split[2].replace('of', '')
    
    distances.append(distance_nm[0:])
    bearings.append(bearing_abrev)

# Create new DataFrame for uas locations
new_df = pd.DataFrame(uas_air_loc['REMARKS'])
new_df['UAS_LOC'] = uas_air_loc['UASLOCATION']
new_df['IDENT'] = uas_air_loc['IDENT']
new_df['Distance'] = distances
new_df['Bearing_Degrees'] = bearings

new_df

,REMARKS,UAS_LOC,IDENT,Distance,Bearing_Degrees
0,"Small uas on the BWI185025, 7,500 feet, headin...",025 NM 185 of BWI,BWI,025,185
1,1756HELO departed CHS as VRTICL3. 1800HELOdepa...,008 NM 130 of CHS,CHS,008,130
2,"uas: Pilot reported a red uas at altitude 5,50...",028 NM 180 of DCA,DCA,028,180
3,"PIC reported a uas, black, 2-3 feet in diamete...",026 NM 267 of DCA,DCA,026,267
4,uas operations DCA271012. Operations terminate...,012 NM 271 of DCA,DCA,012,271
...,...,...,...,...,...
136,DCA-1: uas observed at the DCA283013 just insi...,013 NM 283 of DCA,DCA,013,283
137,"DCA-1: DCA355001, aircraft heading S Ebound re...",001 NM 355 of DCA,DCA,001,355
138,DCA-1: ADW Base Police observed a uas with red...,009 NM 121 of DCA,DCA,009,121
139,DCA-1: uas observed over Madison Building (DCA...,002 NM 052 of DCA,DCA,002,052


In [28]:
# Convert distances from degrees to kilometers in order to be used in geopy later
new_df['Distance'] = pd.to_numeric(new_df['Distance'],errors = 'coerce')
dist_kilo = []

for i in range(len(new_df)):
    distanceKilo = new_df['Distance'][i] * 1.852 
    dist_kilo.append(distanceKilo)


new_df['Distance_Kilometers'] = dist_kilo
new_df['Airport_Latitude'] = uas_air_loc['LATITUDE']
new_df['Airport_Longitude'] = uas_air_loc['LONGITUDE']
new_df

,REMARKS,UAS_LOC,IDENT,Distance,Bearing_Degrees,Distance_Kilometers,Airport_Latitude,Airport_Longitude
0,"Small uas on the BWI185025, 7,500 feet, headin...",025 NM 185 of BWI,BWI,25,185,46.300,39.175728,-76.668991
1,1756HELO departed CHS as VRTICL3. 1800HELOdepa...,008 NM 130 of CHS,CHS,8,130,14.816,32.898639,-80.040528
2,"uas: Pilot reported a red uas at altitude 5,50...",028 NM 180 of DCA,DCA,28,180,51.856,38.851444,-77.037722
3,"PIC reported a uas, black, 2-3 feet in diamete...",026 NM 267 of DCA,DCA,26,267,48.152,38.851444,-77.037722
4,uas operations DCA271012. Operations terminate...,012 NM 271 of DCA,DCA,12,271,22.224,38.851444,-77.037722
...,...,...,...,...,...,...,...,...
136,DCA-1: uas observed at the DCA283013 just insi...,013 NM 283 of DCA,DCA,13,283,24.076,38.851444,-77.037722
137,"DCA-1: DCA355001, aircraft heading S Ebound re...",001 NM 355 of DCA,DCA,1,355,1.852,38.851444,-77.037722
138,DCA-1: ADW Base Police observed a uas with red...,009 NM 121 of DCA,DCA,9,121,16.668,38.851444,-77.037722
139,DCA-1: uas observed over Madison Building (DCA...,002 NM 052 of DCA,DCA,2,052,3.704,38.851444,-77.037722


In [29]:
# Drop unnecessary columns and rename others to keep
new_df2 = pd.merge(new_df, airports, on='IDENT', how='left')
new_df2 = new_df2.drop(['Airport_Latitude', 'Airport_Longitude'], 1)
new_df2 = new_df2.rename({'LONGITUDE': 'Airport_Longitude', 'LATITUDE': 'Airport_Latitude'}, axis=1)
new_df2

,REMARKS,UAS_LOC,IDENT,Distance,Bearing_Degrees,Distance_Kilometers,NAME,Airport_Latitude,Airport_Longitude,ICAO_ID
0,"Small uas on the BWI185025, 7,500 feet, headin...",025 NM 185 of BWI,BWI,25,185,46.300,Baltimore/Washington Intl Thurgood Marshall,39.175728,-76.668991,KBWI
1,1756HELO departed CHS as VRTICL3. 1800HELOdepa...,008 NM 130 of CHS,CHS,8,130,14.816,Charleston AFB/Intl,32.898639,-80.040528,KCHS
2,"uas: Pilot reported a red uas at altitude 5,50...",028 NM 180 of DCA,DCA,28,180,51.856,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA
3,"PIC reported a uas, black, 2-3 feet in diamete...",026 NM 267 of DCA,DCA,26,267,48.152,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA
4,uas operations DCA271012. Operations terminate...,012 NM 271 of DCA,DCA,12,271,22.224,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA
...,...,...,...,...,...,...,...,...,...,...
136,DCA-1: uas observed at the DCA283013 just insi...,013 NM 283 of DCA,DCA,13,283,24.076,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA
137,"DCA-1: DCA355001, aircraft heading S Ebound re...",001 NM 355 of DCA,DCA,1,355,1.852,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA
138,DCA-1: ADW Base Police observed a uas with red...,009 NM 121 of DCA,DCA,9,121,16.668,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA
139,DCA-1: uas observed over Madison Building (DCA...,002 NM 052 of DCA,DCA,2,052,3.704,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA


In [30]:
# Calculate the UAS latitude and longitude using geopy
uas_lat = []
uas_long = []

for i in range(len(new_df)):
    lat_airport = pd.to_numeric(new_df2['Airport_Latitude'][i])
    long_airport = pd.to_numeric(new_df2['Airport_Longitude'][i])
    b = pd.to_numeric(new_df['Bearing_Degrees'][i])
    d = pd.to_numeric(new_df['Distance_Kilometers'][i])
    
    origin = geopy.Point(lat_airport, long_airport)
    destination = geodesic(kilometers=d).destination(origin,b)
    
    lat2, lon2, = destination.latitude, destination.longitude
    
    uas_lat.append(lat2)
    uas_long.append(lon2)

new_df2['UAS_Latitude'] = uas_lat
new_df2['UAS_Longitude'] = uas_long
new_df2

,REMARKS,UAS_LOC,IDENT,Distance,Bearing_Degrees,Distance_Kilometers,NAME,Airport_Latitude,Airport_Longitude,ICAO_ID,UAS_Latitude,UAS_Longitude
0,"Small uas on the BWI185025, 7,500 feet, headin...",025 NM 185 of BWI,BWI,25,185,46.300,Baltimore/Washington Intl Thurgood Marshall,39.175728,-76.668991,KBWI,38.760245,-76.715417
1,1756HELO departed CHS as VRTICL3. 1800HELOdepa...,008 NM 130 of CHS,CHS,8,130,14.816,Charleston AFB/Intl,32.898639,-80.040528,KCHS,32.812707,-79.919335
2,"uas: Pilot reported a red uas at altitude 5,50...",028 NM 180 of DCA,DCA,28,180,51.856,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA,38.384308,-77.037722
3,"PIC reported a uas, black, 2-3 feet in diamete...",026 NM 267 of DCA,DCA,26,267,48.152,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA,38.827431,-77.591479
4,uas operations DCA271012. Operations terminate...,012 NM 271 of DCA,DCA,12,271,22.224,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA,38.854658,-77.293711
...,...,...,...,...,...,...,...,...,...,...,...,...
136,DCA-1: uas observed at the DCA283013 just insi...,013 NM 283 of DCA,DCA,13,283,24.076,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA,38.899918,-77.308148
137,"DCA-1: DCA355001, aircraft heading S Ebound re...",001 NM 355 of DCA,DCA,1,355,1.852,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA,38.868064,-77.039582
138,DCA-1: ADW Base Police observed a uas with red...,009 NM 121 of DCA,DCA,9,121,16.668,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA,38.773998,-76.873314
139,DCA-1: uas observed over Madison Building (DCA...,002 NM 052 of DCA,DCA,2,052,3.704,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA,38.871981,-77.004089


In [31]:
# Export the dataframe into csv file
new_df2.to_csv('COMPLETED-Non-std-141Airports(FR).csv', index = False)

In [ ]:
#################################################################################################################

In [33]:
# Read in the unmatched records to check if they are NAVAID
uas_nav_loc = pd.read_csv('C:/Users/Olga/Desktop/GMU/DAEN690/LEGO/7FixedRadial.csv')
uas_nav_loc = uas_nav_loc.drop(['NAME', 'LATITUDE', 'LONGITUDE', 'ICAO_ID'], axis=1)
uas_nav_loc

,IDENT,REMARKS,UASLOCATION
0,CFR,DCA-1: White House at condition Yellow. Report...,586 NM 100 of CFR
1,SIE,PIC reported uas off it's left wing at same al...,012 NM 330 of SIE
2,VAN,PIC reported uas off left side at FL250. No ev...,045 NM 091 of VAN
3,BRV,Notified that Stafford County Sheriff's Office...,007 NM 005 of BRV
4,LIN,"2056Z, FDC NOTAM 0/1240, The Quarter Fire TFR,...",033 NM 072 of LIN
5,XCH,"Navy Yard, DC reports a uas detection only ins...",030 NM 940 of XCH
6,XEB,Navy Yard reports detection systems at NSF Arl...,201 NM 100 of XEB


In [34]:
# Read in the NAVAID dataset
navaid = pd.read_csv('C:/Users/Olga/Desktop/GMU/DAEN690/LEGO/NAVAID_Cleaned.csv')
navaid

,LONGITUDE,LATITUDE,IDENT,NAME_TXT
0,-176.676024,51.871896,ADK,MOUNT MOFFETT
1,-151.744257,68.136626,AKP,ANAKTUVUK PASS
2,-149.960164,61.167868,TED,ANCHORAGE
3,-149.769204,61.255004,EDF,ELMENDORF
4,-159.597931,61.590169,ANI,ANIAK
...,...,...,...,...
1669,-156.788152,71.273426,BRW,BARROW
1670,-156.781302,71.282300,VIR,BROWERVILLE
1671,-75.164524,43.026523,UCA,UTICA
1672,-90.434470,40.937351,GBG,GALESBURG


In [35]:
# Merge the records with the NAVAID dataset
join_uas_nav = pd.merge(uas_nav_loc, navaid, on='IDENT', how='left')
join_uas_nav

,IDENT,REMARKS,UASLOCATION,LONGITUDE,LATITUDE,NAME_TXT
0,CFR,DCA-1: White House at condition Yellow. Report...,586 NM 100 of CFR,NaN,NaN,NaN
1,SIE,PIC reported uas off it's left wing at same al...,012 NM 330 of SIE,-74.800345,39.095517,SEA ISLE
2,VAN,PIC reported uas off left side at FL250. No ev...,045 NM 091 of VAN,-80.448646,33.474837,VANCE
3,BRV,Notified that Stafford County Sheriff's Office...,007 NM 005 of BRV,-77.352868,38.336265,BROOKE
4,LIN,"2056Z, FDC NOTAM 0/1240, The Quarter Fire TFR,...",033 NM 072 of LIN,-121.003871,38.074593,LINDEN
5,XCH,"Navy Yard, DC reports a uas detection only ins...",030 NM 940 of XCH,NaN,NaN,NaN
6,XEB,Navy Yard reports detection systems at NSF Arl...,201 NM 100 of XEB,NaN,NaN,NaN


In [36]:
# Find the actual NAVAID records
nav_loc = join_uas_nav[join_uas_nav['NAME_TXT'].notnull()]
nav_loc

,IDENT,REMARKS,UASLOCATION,LONGITUDE,LATITUDE,NAME_TXT
1,SIE,PIC reported uas off it's left wing at same al...,012 NM 330 of SIE,-74.800345,39.095517,SEA ISLE
2,VAN,PIC reported uas off left side at FL250. No ev...,045 NM 091 of VAN,-80.448646,33.474837,VANCE
3,BRV,Notified that Stafford County Sheriff's Office...,007 NM 005 of BRV,-77.352868,38.336265,BROOKE
4,LIN,"2056Z, FDC NOTAM 0/1240, The Quarter Fire TFR,...",033 NM 072 of LIN,-121.003871,38.074593,LINDEN


In [37]:
df = pd.merge(nav_loc, nonstd, on='REMARKS', how='left')
df

,IDENT,REMARKS,UASLOCATION_x,LONGITUDE,LATITUDE,NAME_TXT,dateonly,timeonly,DATE,CALLSIGN,...,ACALTITUDE.CEDAR,ACHEADING.CEDAR,RELATIVECLOCKPOSITION.CEDAR,UASLONG.CEDAR,UASLAT.CEDAR,SUMMARY.CEDAR,QAFINDINGS.CEDAR,RWYLOCATION,UASLOCATION_y,DATASET
0,SIE,PIC reported uas off it's left wing at same al...,012 NM 330 of SIE,-74.800345,39.095517,SEA ISLE,7/7/2018,23:33,2018-07-07T23:33Z,PDT4865,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,012 NM 330 of SIE,FIXED RADIAL
1,VAN,PIC reported uas off left side at FL250. No ev...,045 NM 091 of VAN,-80.448646,33.474837,VANCE,12/6/2018,20:29,2018-12-06T20:29Z,UAL356,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,045 NM 091 of VAN,FIXED RADIAL
2,BRV,Notified that Stafford County Sheriff's Office...,007 NM 005 of BRV,-77.352868,38.336265,BROOKE,7/1/2019,20:32,2019-07-01T20:32Z,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,007 NM 005 of BRV,FIXED RADIAL
3,LIN,"2056Z, FDC NOTAM 0/1240, The Quarter Fire TFR,...",033 NM 072 of LIN,-121.003871,38.074593,LINDEN,6/23/2020,21:25,2020-06-23T21:25Z,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,033 NM 072 of LIN,FIXED RADIAL


In [38]:
# add UASLOCATION to the dataframe nav_loc
add_col1 = df['UASLOCATION_y'].tolist()
nav_loc['UASLOCATION'] = add_col1
nav_loc

<ipython-input-38-f6a078998b6c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nav_loc['UASLOCATION'] = add_col1


,IDENT,REMARKS,UASLOCATION,LONGITUDE,LATITUDE,NAME_TXT
1,SIE,PIC reported uas off it's left wing at same al...,012 NM 330 of SIE,-74.800345,39.095517,SEA ISLE
2,VAN,PIC reported uas off left side at FL250. No ev...,045 NM 091 of VAN,-80.448646,33.474837,VANCE
3,BRV,Notified that Stafford County Sheriff's Office...,007 NM 005 of BRV,-77.352868,38.336265,BROOKE
4,LIN,"2056Z, FDC NOTAM 0/1240, The Quarter Fire TFR,...",033 NM 072 of LIN,-121.003871,38.074593,LINDEN


In [46]:
nav_loc.to_csv('4NAVAID.csv', index = False)

nav_loc = pd.read_csv('C:/Users/Olga/Desktop/GMU/DAEN690/LEGO/4NAVAID.csv')

In [47]:
# Separate the distance and bearing into two different columns
uas_loc = nav_loc['UASLOCATION']
distances = []
bearings = []

for i in range(len(uas_loc)):
    split = uas_loc[i].split(' ')[0:3]
    
    if 'NM' in split[0]: 
        distance_nm = split[0][0:len(split[0]) - 2]
        bearing_abrev = split[1].replace('of', '')
        
    else: 
        distance_nm = split[0]
        bearing_abrev = split[2].replace('of', '')
    
    distances.append(float(distance_nm))
    bearings.append(bearing_abrev)
    

# Create new DataFrame for uas locations
new_df = pd.DataFrame(nav_loc['REMARKS'])
new_df['UASLOCATION'] = nav_loc['UASLOCATION']
new_df['IDENT'] = nav_loc['IDENT']
new_df['Distance'] = distances
new_df['Bearing_Degrees'] = bearings
new_df['NAME_TXT'] = nav_loc['NAME_TXT']

new_df

,REMARKS,UASLOCATION,IDENT,Distance,Bearing_Degrees,NAME_TXT
0,PIC reported uas off it's left wing at same al...,012 NM 330 of SIE,SIE,12.0,330,SEA ISLE
1,PIC reported uas off left side at FL250. No ev...,045 NM 091 of VAN,VAN,45.0,091,VANCE
2,Notified that Stafford County Sheriff's Office...,007 NM 005 of BRV,BRV,7.0,005,BROOKE
3,"2056Z, FDC NOTAM 0/1240, The Quarter Fire TFR,...",033 NM 072 of LIN,LIN,33.0,072,LINDEN


In [48]:
# Find all distances in kilometers
dist_kilo = []

for i in range(len(new_df)):
    distanceKilo = new_df['Distance'][i] * 1.852 # converting NM to kilometers
    dist_kilo.append(distanceKilo)

new_df['Distance_Kilometers'] = dist_kilo
new_df['NAVAID_Latitude'] = nav_loc['LATITUDE']
new_df['NAVAID_Longitude'] = nav_loc['LONGITUDE']
new_df['NAME_TXT'] = nav_loc['NAME_TXT']
new_df

,REMARKS,UASLOCATION,IDENT,Distance,Bearing_Degrees,NAME_TXT,Distance_Kilometers,NAVAID_Latitude,NAVAID_Longitude
0,PIC reported uas off it's left wing at same al...,012 NM 330 of SIE,SIE,12.0,330,SEA ISLE,22.224,39.095517,-74.800345
1,PIC reported uas off left side at FL250. No ev...,045 NM 091 of VAN,VAN,45.0,091,VANCE,83.340,33.474837,-80.448646
2,Notified that Stafford County Sheriff's Office...,007 NM 005 of BRV,BRV,7.0,005,BROOKE,12.964,38.336265,-77.352868
3,"2056Z, FDC NOTAM 0/1240, The Quarter Fire TFR,...",033 NM 072 of LIN,LIN,33.0,072,LINDEN,61.116,38.074593,-121.003871


In [49]:
# Calculate UAS latitude and Longitude
uas_lat = []
uas_long = []


for i in range(len(new_df)):
    lat_nav = pd.to_numeric(new_df['NAVAID_Latitude'][i])
    long_nav = pd.to_numeric(new_df['NAVAID_Longitude'][i])
    b = pd.to_numeric(new_df['Bearing_Degrees'][i])
    d = pd.to_numeric(new_df['Distance_Kilometers'][i])
    
    origin = geopy.Point(lat_nav, long_nav)
    destination = geodesic(kilometers=d).destination(origin,b)
    
    lat2, lon2, = destination.latitude, destination.longitude
    
    uas_lat.append(lat2)
    uas_long.append(lon2)
    
new_df['UAS_Latitude'] = uas_lat
new_df['UAS_Longitude'] = uas_long


new_df

,REMARKS,UASLOCATION,IDENT,Distance,Bearing_Degrees,NAME_TXT,Distance_Kilometers,NAVAID_Latitude,NAVAID_Longitude,UAS_Latitude,UAS_Longitude
0,PIC reported uas off it's left wing at same al...,012 NM 330 of SIE,SIE,12.0,330,SEA ISLE,22.224,39.095517,-74.800345,39.268808,-74.929109
1,PIC reported uas off left side at FL250. No ev...,045 NM 091 of VAN,VAN,45.0,091,VANCE,83.340,33.474837,-80.448646,33.458482,-79.552323
2,Notified that Stafford County Sheriff's Office...,007 NM 005 of BRV,BRV,7.0,005,BROOKE,12.964,38.336265,-77.352868,38.452608,-77.339924
3,"2056Z, FDC NOTAM 0/1240, The Quarter Fire TFR,...",033 NM 072 of LIN,LIN,33.0,072,LINDEN,61.116,38.074593,-121.003871,38.242864,-120.339902


In [50]:
# Export the dataframe into csv file
new_df.to_csv('COMPLETED-Non-std-4NAVAID(FR).csv', index = False)

In [ ]:
#####################################################################################################################

In [51]:
# Find the final unmatched records and save them into separate dataframe for further investigation 
remaining = join_uas_nav[join_uas_nav['NAME_TXT'].isnull()]
remaining = remaining.drop(['NAME_TXT', 'LATITUDE', 'LONGITUDE'], axis=1)
remaining

,IDENT,REMARKS,UASLOCATION
0,CFR,DCA-1: White House at condition Yellow. Report...,586 NM 100 of CFR
5,XCH,"Navy Yard, DC reports a uas detection only ins...",030 NM 940 of XCH
6,XEB,Navy Yard reports detection systems at NSF Arl...,201 NM 100 of XEB


In [52]:
# Export dataframe into csv file
remaining.to_csv('Non-std-3Remaining.csv', index = False)

In [ ]:
######################################################################################################################

In [53]:
# Read in CSV file
nonstd1 = pd.read_csv('C:/Users/Olga/Desktop/GMU/DAEN690/LEGO/Non-std-3FixedRadial.csv')
nonstd1

,IDENT,REMARKS,OLD_UASLOCATION,dateonly,timeonly,DATE,CALLSIGN,POD,PRIMARYCODE,SECONDARYCODES,...,ACALTITUDE.CEDAR,ACHEADING.CEDAR,RELATIVECLOCKPOSITION.CEDAR,UASLONG.CEDAR,UASLAT.CEDAR,SUMMARY.CEDAR,QAFINDINGS.CEDAR,RWYLOCATION,DATASET,NEW_UASLOCATION
0,RCC,"DCA-1: Incident occurred at 1835 (1335L), firs...",675 NM N NCRCC,2/16/2019,2:36,2019-02-16T02:36Z,NaN,NCRCC,UAS,LAW ENFORCEMENT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FIXED RADIAL DISTANCE,3.5 NM 300 of DCA
1,USCP,TOI P-56A// DCA-1: USCP reported a MPD officer...,1100 NM N USCP,1/20/2021,15:25,2021-01-20T15:25Z,NaN,NCRCC,TOI-PA-P-56A,UAS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FIXED RADIAL DISTANCE,3 NM 036 of DCA
2,FED,"DCA-1: DCA072008, 1 NW of FEDEX Field, MUSEL18...",1 NM NW of FED,6/21/2021,18:56,2021-06-21T18:56Z,MUSEL18,NCRCC,UAS,"SUSPICIOUS ACTIVITY, TOI-NCR FRZ",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FIXED RADIAL DISTANCE,8 NM 072 of DCA


In [54]:
# Extract IDENTs from the UAS location column
uas_loc = nonstd1['NEW_UASLOCATION'].tolist()
uas_airport = []

for i in range(len(uas_loc)):
    airport = uas_loc[i].split(' ')[-1]
    
    if len(airport) <= 4:
        uas_airport.append(airport)
    else:
        trim_air = airport[-3:]
        uas_airport.append(trim_air)
        
uas_airport_df = pd.DataFrame()
uas_airport_df['IDENT'] = uas_airport
uas_airport_df

,IDENT
0,DCA
1,DCA
2,DCA


In [55]:
# Merge the IDENTs dataframe with airports dataset to get lat/long coordinates
uas_air_loc= pd.merge(uas_airport_df, airports, on='IDENT', how='left')
uas_air_loc

,IDENT,NAME,LATITUDE,LONGITUDE,ICAO_ID
0,DCA,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA
1,DCA,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA
2,DCA,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA


In [56]:
# Add REMARKS and UAS location columns to merged dataframe
add_col2 = nonstd1['REMARKS'].tolist()
uas_air_loc['REMARKS'] = add_col2
add_col3 = nonstd1['NEW_UASLOCATION'].tolist()
uas_air_loc['UASLOCATION'] = add_col3
uas_air_loc

,IDENT,NAME,LATITUDE,LONGITUDE,ICAO_ID,REMARKS,UASLOCATION
0,DCA,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA,"DCA-1: Incident occurred at 1835 (1335L), firs...",3.5 NM 300 of DCA
1,DCA,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA,TOI P-56A// DCA-1: USCP reported a MPD officer...,3 NM 036 of DCA
2,DCA,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA,"DCA-1: DCA072008, 1 NW of FEDEX Field, MUSEL18...",8 NM 072 of DCA


In [57]:
# Separate distance and bearing into two different columns
uas_loc = uas_air_loc['UASLOCATION']
distances = []
bearings = []

for i in range(len(uas_loc)):
    split = uas_loc[i].split(' ')[0:3]
    
    if 'NM' in split[0]: #if there is no space between the distance and NM (i.e. 3NM)
        distance_nm = split[0][0:len(split[0]) - 2]
        bearing_abrev = split[1].replace('of', '')
        
    else: 
        distance_nm = split[0]
        bearing_abrev = split[2].replace('of', '')
    
    distances.append(distance_nm[0:])
    bearings.append(bearing_abrev)

# Create UAS location dataframe
new_df = pd.DataFrame(uas_air_loc['REMARKS'])
new_df['UAS_LOC'] = uas_air_loc['UASLOCATION']
new_df['IDENT'] = uas_air_loc['IDENT']
new_df['Distance'] = distances
new_df['Bearing_Degrees'] = bearings

new_df

,REMARKS,UAS_LOC,IDENT,Distance,Bearing_Degrees
0,"DCA-1: Incident occurred at 1835 (1335L), firs...",3.5 NM 300 of DCA,DCA,3.5,300
1,TOI P-56A// DCA-1: USCP reported a MPD officer...,3 NM 036 of DCA,DCA,3,036
2,"DCA-1: DCA072008, 1 NW of FEDEX Field, MUSEL18...",8 NM 072 of DCA,DCA,8,072


In [58]:
# Converting distances to kilometers
new_df['Distance'] = pd.to_numeric(new_df['Distance'],errors = 'coerce')
dist_kilo = []

for i in range(len(new_df)):
    distanceKilo = new_df['Distance'][i] * 1.852 
    dist_kilo.append(distanceKilo)

new_df['Distance_Kilometers'] = dist_kilo
new_df['Airport_Latitude'] = uas_air_loc['LATITUDE']
new_df['Airport_Longitude'] = uas_air_loc['LONGITUDE']
new_df

,REMARKS,UAS_LOC,IDENT,Distance,Bearing_Degrees,Distance_Kilometers,Airport_Latitude,Airport_Longitude
0,"DCA-1: Incident occurred at 1835 (1335L), firs...",3.5 NM 300 of DCA,DCA,3.5,300,6.482,38.851444,-77.037722
1,TOI P-56A// DCA-1: USCP reported a MPD officer...,3 NM 036 of DCA,DCA,3.0,036,5.556,38.851444,-77.037722
2,"DCA-1: DCA072008, 1 NW of FEDEX Field, MUSEL18...",8 NM 072 of DCA,DCA,8.0,072,14.816,38.851444,-77.037722


In [59]:
# Drop unnecessary columns and rename needed ones
new_df2 = pd.merge(new_df, airports, on='IDENT', how='left')
new_df2 = new_df2.drop(['Airport_Latitude', 'Airport_Longitude'], 1)
new_df2 = new_df2.rename({'LONGITUDE': 'Airport_Longitude', 'LATITUDE': 'Airport_Latitude'}, axis=1)
new_df2

,REMARKS,UAS_LOC,IDENT,Distance,Bearing_Degrees,Distance_Kilometers,NAME,Airport_Latitude,Airport_Longitude,ICAO_ID
0,"DCA-1: Incident occurred at 1835 (1335L), firs...",3.5 NM 300 of DCA,DCA,3.5,300,6.482,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA
1,TOI P-56A// DCA-1: USCP reported a MPD officer...,3 NM 036 of DCA,DCA,3.0,036,5.556,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA
2,"DCA-1: DCA072008, 1 NW of FEDEX Field, MUSEL18...",8 NM 072 of DCA,DCA,8.0,072,14.816,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA


In [60]:
# Calculate UAS lat/long coordinates
uas_lat = []
uas_long = []

for i in range(len(new_df)):
    lat_airport = pd.to_numeric(new_df2['Airport_Latitude'][i])
    long_airport = pd.to_numeric(new_df2['Airport_Longitude'][i])
    b = pd.to_numeric(new_df['Bearing_Degrees'][i])
    d = pd.to_numeric(new_df['Distance_Kilometers'][i])
    
    origin = geopy.Point(lat_airport, long_airport)
    destination = geodesic(kilometers=d).destination(origin,b)
    
    lat2, lon2, = destination.latitude, destination.longitude
    
    uas_lat.append(lat2)
    uas_long.append(lon2)

# Append to UAS locations dataframe
new_df2['UAS_Latitude'] = uas_lat
new_df2['UAS_Longitude'] = uas_long
new_df2

,REMARKS,UAS_LOC,IDENT,Distance,Bearing_Degrees,Distance_Kilometers,NAME,Airport_Latitude,Airport_Longitude,ICAO_ID,UAS_Latitude,UAS_Longitude
0,"DCA-1: Incident occurred at 1835 (1335L), firs...",3.5 NM 300 of DCA,DCA,3.5,300,6.482,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA,38.880621,-77.102416
1,TOI P-56A// DCA-1: USCP reported a MPD officer...,3 NM 036 of DCA,DCA,3.0,036,5.556,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA,38.891928,-77.000080
2,"DCA-1: DCA072008, 1 NW of FEDEX Field, MUSEL18...",8 NM 072 of DCA,DCA,8.0,072,14.816,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA,38.892574,-76.875305


In [61]:
# Export dataframe to csv file
new_df2.to_csv('COMPLETED-Non-std-3Airports(FR).csv', index = False)

In [ ]:
######################################################################################################################